In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Convolution1D, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import plot_confusion_matrix
from warnings import simplefilter
import time
start = time.time()

In [3]:
df = pd.read_csv('attack_all.csv')
seed = 7
class_names = df.target.unique()
df=df.astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
y = df['target']
x_columns = df.columns.drop('target')
x = df[x_columns].values


print("Ready to generate train and test datasets")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=seed)
print("x_train, y_train, x_test, y_test" + str(x_train.shape) + "" +str(y_train.shape) + "" +str(x_test.shape) + "" +str(y_test.shape))

Ready to generate train and test datasets
x_train, y_train, x_test, y_test(3600, 16)(3600,)(1544, 16)(1544,)


In [4]:
print("Ready to generate train and test datasets")

#Neural network
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(30, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal'))
model.add(Dense(4,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=200,batch_size=1000) 
model.save("Results/predict.h5")
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nn = model.predict(x_test)
y_pred_nn = np.argmax(y_pred_nn,axis=1)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

print(model.summary())

Ready to generate train and test datasets
Epoch 1/200
4/4 - 2s - loss: 7.8716 - accuracy: 0.2633 - val_loss: 3.6860 - val_accuracy: 0.3297 - 2s/epoch - 463ms/step
Epoch 2/200
4/4 - 0s - loss: 3.4580 - accuracy: 0.2867 - val_loss: 3.7871 - val_accuracy: 0.2461 - 104ms/epoch - 26ms/step
Epoch 3/200
4/4 - 0s - loss: 2.8066 - accuracy: 0.2956 - val_loss: 1.5260 - val_accuracy: 0.4566 - 67ms/epoch - 17ms/step
Epoch 4/200
4/4 - 0s - loss: 1.8541 - accuracy: 0.3700 - val_loss: 1.8275 - val_accuracy: 0.4301 - 67ms/epoch - 17ms/step
Epoch 5/200
4/4 - 0s - loss: 1.8336 - accuracy: 0.4589 - val_loss: 1.6264 - val_accuracy: 0.4903 - 52ms/epoch - 13ms/step
Epoch 6/200
4/4 - 0s - loss: 1.4253 - accuracy: 0.4878 - val_loss: 1.2913 - val_accuracy: 0.4093 - 69ms/epoch - 17ms/step
Epoch 7/200
4/4 - 0s - loss: 1.3362 - accuracy: 0.3825 - val_loss: 1.2197 - val_accuracy: 0.4547 - 124ms/epoch - 31ms/step
Epoch 8/200
4/4 - 0s - loss: 1.1784 - accuracy: 0.5228 - val_loss: 1.1539 - val_accuracy: 0.5816 - 108m

Epoch 68/200
4/4 - 0s - loss: 0.4825 - accuracy: 0.8022 - val_loss: 0.5152 - val_accuracy: 0.8083 - 83ms/epoch - 21ms/step
Epoch 69/200
4/4 - 0s - loss: 0.4792 - accuracy: 0.8064 - val_loss: 0.5077 - val_accuracy: 0.8096 - 56ms/epoch - 14ms/step
Epoch 70/200
4/4 - 0s - loss: 0.4719 - accuracy: 0.8169 - val_loss: 0.5138 - val_accuracy: 0.8038 - 82ms/epoch - 20ms/step
Epoch 71/200
4/4 - 0s - loss: 0.4711 - accuracy: 0.8111 - val_loss: 0.5116 - val_accuracy: 0.8102 - 68ms/epoch - 17ms/step
Epoch 72/200
4/4 - 0s - loss: 0.4735 - accuracy: 0.8117 - val_loss: 0.5098 - val_accuracy: 0.8154 - 83ms/epoch - 21ms/step
Epoch 73/200
4/4 - 0s - loss: 0.4775 - accuracy: 0.8122 - val_loss: 0.5400 - val_accuracy: 0.7804 - 94ms/epoch - 23ms/step
Epoch 74/200
4/4 - 0s - loss: 0.4908 - accuracy: 0.7989 - val_loss: 0.5321 - val_accuracy: 0.7759 - 102ms/epoch - 26ms/step
Epoch 74: early stopping
Training time: 11.05800485610962
49/49 [==============================] - 0s 2ms/step
Test time: 0.49221634864807

In [5]:
print("Neural network, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nn,average='weighted')))
matrixnn = confusion_matrix(y_test,y_pred_nn)
print(matrixnn)

Neural network, accuracy: 0.7759067357512953 F1 score:0.7766168582846303
[[127  60  38  17]
 [ 34 376   5  17]
 [128   8 167  12]
 [ 20   4   3 528]]


In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify, render_template
import pandas as pd
from keras.models import load_model
import numpy as np


# Load the pre-trained CNN model
model = load_model('predict.h5')

# Create a Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

# Define a route to handle file uploads
@app.route('/upload', methods=['POST'])
def upload():
    print("hello")
    # Get the uploaded file from the request
    file = request.files['file']

    # Parse the CSV file using Pandas
    df = pd.read_csv(file)
    print(df)
    class_names = df.target.unique()
    df=df.astype('category')
    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    x_columns = df.columns.drop('target')
    x = df[x_columns].values
    df=x.to_csv(index=False)

    # Predict the classes or labels using the CNN model
    classes = model.predict(x)

    # Convert the predictions to a human-readable format
    predicted_labels = np.argmax(classes, axis=1)
    labels = ['version_num' 'legitimate' 'sinkhole' 'dis_flood']
    predicted_classes = [labels[label] for label in predicted_labels]

    # Return the predicted classes as a response
    return jsonify({'predicted_classes': predicted_classes})


# Run the Flask app
if __name__ == '__main__':
    app.run(debug=False)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2023-04-28 12:01:50,507] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Dell\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Dell\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Dell\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Dell\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Dell\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Dell\AppData\Local\